In [188]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
import statsmodels.api as sm
import sklearn.model_selection as skm
import sklearn.linear_model as skl
from sklearn.preprocessing import StandardScaler
from ISLP import load_data
from ISLP.models import ModelSpec as MS
from functools import partial


from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from ISLP.models import \
                (Stepwise,
                sklearn_selected,
                sklearn_selection_path,sklearn_sm)
from l0bnb import fit_path

College = load_data('College')
College = College.dropna()

-- Linear Regression --

In [189]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import sklearn.model_selection as skm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from ISLP.models import sklearn_sm  # if you're using the ISLP package
from ISLP import load_data
from ISLP.models import ModelSpec as MS


# 1. Split data into training (265) and test (512)
College_train, College_valid = skm.train_test_split(College,
                                                    test_size=512,
                                                    random_state=1)

# 2. Set up ModelSpec
vars = College.columns.drop('Apps')
design = MS(vars)

# 3. Create design matrices
X_train = design.fit_transform(College_train)
y_train = College_train['Apps']

X_valid = design.transform(College_valid)
y_valid = College_valid['Apps']

# 4. Fit OLS using statsmodels
model = sm.OLS(y_train, X_train)
results = model.fit()

# 5. Predict on validation set and compute test MSE
valid_pred = results.predict(X_valid)
ols_test_mse = mean_squared_error(y_valid, valid_pred)
print("OLS MSE:", ols_test_mse)

# 6. 5-fold Cross-Validation using ISLP's sklearn wrapper
# Cross validation
hp_model = sklearn_sm(sm.OLS, MS(vars))
X, Y = College.drop(columns=['Apps']), College['Apps']
cv_results = skm.cross_validate(hp_model,X,Y,cv=College.shape[0])
cv_err = np.mean(cv_results['test_score'])


OLS MSE: 1392289.3825379037


-- Linear regression test error --

In [190]:
print("OLS MSE:", ols_test_mse)
print("Cross-validated MSE:", cv_err)

OLS MSE: 1392289.3825379037
Cross-validated MSE: 1276986.7007955108


-- Ridge Regrssion -- 

In [191]:


# Lambda grid
lambdas = 10**np.linspace(8, -2, 100) / y_train.std()

# Scaling and CV setup
scaler = StandardScaler()
inner_cv = skm.KFold(n_splits=5, shuffle=True, random_state=2)

# Ridge regression (l1_ratio=0)
ridgeCV = skl.ElasticNetCV(alphas=lambdas, l1_ratio=0, cv=inner_cv)
pipe_ridge = Pipeline(steps=[('scaler', scaler), ('ridge', ridgeCV)])
pipe_ridge.fit(X_train, y_train)

ridge_pred = pipe_ridge.predict(X_valid)
ridge_test_error = mean_squared_error(y_valid, ridge_pred)







/opt/anaconda3/envs/CSC293/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:681: UserWarning: Coordinate descent without L1 regularization may lead to unexpected results and is discouraged. Set l1_ratio > 0 to add L1 regularization.
  model = cd_fast.enet_coordinate_descent_gram(
/opt/anaconda3/envs/CSC293/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1375484783.8711514, tolerance: 275131.9394886794
  model = cd_fast.enet_coordinate_descent_gram(
/opt/anaconda3/envs/CSC293/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:681: UserWarning: Coordinate descent without L1 regularization may lead to unexpected results and is discouraged. Set l1_ratio > 0 to add L1 regularization.
  model = cd_fast.enet_coordinate_descent_gram(
/opt/anaconda3/envs/CSC293/lib/python3.13/site-packages/sklearn/linear_mod

-- Ridge Regression test error --

In [192]:
ridge_test_error

1572536.590117208

-- Lasso Regression --

In [193]:

# Set up Lasso
scaler = StandardScaler()
lambdas = 10**np.linspace(8, -2, 100) / y_train.std()
inner_cv = skm.KFold(n_splits=5, shuffle=True, random_state=2)

lassoCV = skl.ElasticNetCV(alphas=lambdas, l1_ratio=1, cv=inner_cv)
pipe_lasso = Pipeline(steps=[('scaler', scaler), ('lasso', lassoCV)])
pipe_lasso.fit(X_train, y_train)

# Evaluate on test set
lasso_pred = pipe_lasso.predict(X_valid)
lasso_test_error = mean_squared_error(y_valid, lasso_pred)

# Count non-zero coefficients
tuned_lasso = pipe_lasso.named_steps['lasso']
num_nonzero = np.sum(tuned_lasso.coef_ != 0)




-- Lasso Regression test error --

In [194]:
lasso_test_error

1382689.8391999742

-- Number of non-zero coefficient estimates --

In [195]:
print("Number of non-zero coefficients:", num_nonzero)

Number of non-zero coefficients: 17


-- PCR model --

In [196]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

# Set up PCR model
scaler = StandardScaler()
pca = PCA()
linreg = skl.LinearRegression()
pipe_pcr = Pipeline([
    ('scaler', scaler),
    ('pca', pca),
    ('linreg', linreg)
])

# Cross-validation to choose M (number of components)
param_grid = {'pca__n_components': range(1, X_train.shape[1] + 1)}
kfold=KFold(n_splits=5, shuffle=True, random_state=2)

grid = GridSearchCV(pipe_pcr, param_grid, cv=kfold, scoring='neg_mean_squared_error')
grid.fit(X_train, y_train)

# Evaluate on validation set
best_M = grid.best_params_['pca__n_components']
best_model = grid.best_estimator_
pcr_pred = best_model.predict(X_valid)
pcr_test_error = mean_squared_error(y_valid, pcr_pred)




-- PCR model test error --

In [197]:
print("PCR Test Error:", pcr_test_error)

PCR Test Error: 1447813.5756971166


--PCR M value--

In [198]:
print("Best number of components (M):", best_M)

Best number of components (M): 16


-- PLS model --

In [199]:
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Set up PLS pipeline
scaler = StandardScaler()
pls = PLSRegression()

pipe_pls = Pipeline([
    ('scaler', scaler),
    ('pls', pls)
])

# Cross-validation to choose M (number of components)
param_grid = {'pls__n_components': range(1, X_train.shape[1] + 1)}
kfold = KFold(n_splits=5, shuffle=True, random_state=2)

grid = GridSearchCV(pipe_pls, param_grid, cv=kfold, scoring='neg_mean_squared_error')
grid.fit(X_train, y_train)

# Evaluate on validation set
best_M = grid.best_params_['pls__n_components']
best_model = grid.best_estimator_
pls_pred = best_model.predict(X_valid)
pls_test_error = mean_squared_error(y_valid, pls_pred)

# Output




-- PLS test error --

In [200]:
print("PLS Test MSE:", pls_test_error)

PLS Test MSE: 1423264.6297235435


--PLS M value--

In [201]:
print("Best number of components (M):", best_M)

Best number of components (M): 9


In [202]:
print(cv_err)
print(ridge_test_error)
print(lasso_test_error)
print(pcr_test_error)
print(pls_test_error)

1276986.7007955108
1572536.590117208
1382689.8391999742
1447813.5756971166
1423264.6297235435


In [203]:
np.mean(y_valid)
ols_rmse = np.sqrt(cv_err)
ridge_rmse = np.sqrt(1572536.59) 
lasso_rmse = np.sqrt(1382689.84) 
pcr_rmse   = np.sqrt(1447813.58) 
pls_rmse   = np.sqrt(1423264.63) 
#Percentage Error
ols_perc_error = (ols_rmse / np.mean(y_valid)) * 100
ridge_perc_error = (ridge_rmse / np.mean(y_valid)) * 100
lasso_perc_error = (lasso_rmse / np.mean(y_valid)) * 100
pcr_perc_error   = (pcr_rmse / np.mean(y_valid)) * 100
pls_perc_error   = (pls_rmse / np.mean(y_valid)) * 100
# Print results
print(f"OLS RMSE: {ols_rmse:.2f}, Percentage Error: {ols_perc_error:.2f}%")
print(f"Ridge RMSE: {ridge_rmse:.2f}, Percentage Error: {ridge_perc_error:.2f}%")
print(f"Lasso RMSE: {lasso_rmse:.2f}, Percentage Error: {lasso_perc_error:.2f}%")
print(f"PCR RMSE: {pcr_rmse:.2f}, Percentage Error: {pcr_perc_error:.2f}%")
print(f"PLS RMSE: {pls_rmse:.2f}, Percentage Error: {pls_perc_error:.2f}%")



OLS RMSE: 1130.04, Percentage Error: 38.72%
Ridge RMSE: 1254.01, Percentage Error: 42.97%
Lasso RMSE: 1175.88, Percentage Error: 40.29%
PCR RMSE: 1203.25, Percentage Error: 41.23%
PLS RMSE: 1193.01, Percentage Error: 40.88%


**How accurately can we predict the number of college applications received?**

Based on the test RMSE values, we can predict the number of applications with moderate accuracy. The best-performing model (OLS) had an RMSE of about 1130, which corresponds to a 38.72% error relative to the average number of applications. Other models, including Ridge, Lasso, PCR, and PLS, also performed reasonably well, all falling in the 40–43% range. This suggests that while our models capture general trends in application counts, they are not highly precise and still leave room for improvement.



**Is there much difference among the test errors resulting from these five approaches?**

The test errors from the five approaches are somewhat close

